In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-08-23 03:07:39--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.09s   

2021-08-23 03:07:39 (11.1 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url_list ="https://ucb-data-group-project-zillow-s3.s3.us-west-1.amazonaws.com/final_census_data.csv"
spark.sparkContext.addFile(url_list)
census_data_df = spark.read.csv(SparkFiles.get("final_census_data.csv"), sep=",", header=True, inferSchema=True)

In [5]:
# Show DataFrame
census_data_df.show()

+--------------------+-------------------+-------+------------------+----------+----------+
|         region_date|               city|  state|         sum_level|      date|population|
+--------------------+-------------------+-------+------------------+----------+----------+
|alabamaalabama201...|            Alabama|Alabama|             State|2010-07-01|   4785514|
|abbevillecityalab...|     Abbeville city|Alabama|Incorporated place|2010-07-01|      2699|
|adamsvillecityala...|    Adamsville city|Alabama|Incorporated place|2010-07-01|      4481|
|addisontownalabam...|       Addison town|Alabama|Incorporated place|2010-07-01|       751|
|akrontownalabama2...|         Akron town|Alabama|Incorporated place|2010-07-01|       355|
|alabastercityalab...|     Alabaster city|Alabama|Incorporated place|2010-07-01|     31192|
|albertvillecityal...|   Albertville city|Alabama|Incorporated place|2010-07-01|     21185|
|alexandercitycity...|Alexander City city|Alabama|Incorporated place|2010-07-01|

In [6]:
# Store environmental variable
from getpass import getpass
password = getpass('Enter database password')
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://ucb-data-group-project-zillow.csaw135fqqkl.us-west-1.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres",
          "password": password,
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [7]:
# Write DataFrame to census table in RDS
census_data_df.write.jdbc(url=jdbc_url, table='census', mode=mode, properties=config)